In [ ]:
# This command is using the pip package manager to install several Python packages: langchain, langchain_openai, langchain_community, langgraph, ipykernel, and python-dotenv. These packages are being installed in the current Python environment for use in the project or application.
!pip install langchain langchain_openai langchain_community langgraph ipykernel python-dotenv 

In [ ]:
pip install langchain-huggingface

In [1]:
import cred

In [2]:
huggingface_api_key=cred.hg

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv('huggingface_api_key')

In [4]:
from langchain_huggingface import HuggingFaceEndpoint
llm = HuggingFaceEndpoint(huggingfacehub_api_token=huggingface_api_key,model="Qwen/Qwen2.5-Coder-32B-Instruct")
question = "Is Messi the best footballer of all time?"
output = llm.invoke(question)
print(output[:75])

e:\Projects\GEN_AI\assist_ai_with_langchain\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


 This is a highly debated topic among football fans and experts alike. Whil


In [5]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_core.messages import HumanMessage, AIMessageChunk, SystemMessage

llm = HuggingFaceEndpoint(
    repo_id="Qwen/Qwen2.5-Coder-32B-Instruct",
    task="text-generation",
    max_new_tokens=512,
    do_sample=False,
    repetition_penalty=1.03,
)

chat_model = ChatHuggingFace(llm=llm)
messages = [SystemMessage(content='You are a grumpy pirate.'),
           HumanMessage(content="What's up?")]
output=chat_model.invoke(messages)

In [6]:
type(output)

langchain_core.messages.ai.AIMessage

In [7]:
output.content

"Arrr, th' sky be clear but me head be full o' grumps! What be ye askin' a grumpy pirate like meself? Ye best keep yer jestin' to a minimum if ye value yer petticoat, lass or lad!"

In [8]:
output.dict()

C:\Users\dell nwc\AppData\Local\Temp\ipykernel_6528\436916316.py:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  output.dict()


{'content': "Arrr, th' sky be clear but me head be full o' grumps! What be ye askin' a grumpy pirate like meself? Ye best keep yer jestin' to a minimum if ye value yer petticoat, lass or lad!",
 'additional_kwargs': {},
 'response_metadata': {'token_usage': {'completion_tokens': 57,
   'prompt_tokens': 24,
   'total_tokens': 81},
  'model': '',
  'finish_reason': 'stop'},
 'type': 'ai',
 'name': None,
 'id': 'run-fe4fc134-4559-4208-a60e-7b927d3bc443-0',
 'example': False,
 'tool_calls': [],
 'invalid_tool_calls': [],
 'usage_metadata': None}

In [9]:
from langchain_core.prompts import PromptTemplate
query_template = "Tell me about {book_name} by {author}."
prompt = PromptTemplate(input_variables=["book_name", "author"], template=query_template)
prompt.invoke({"book_name": "Song of Ice and Fire", "author": "GRRM"})

StringPromptValue(text='Tell me about Song of Ice and Fire by GRRM.')

In [10]:

llm = HuggingFaceEndpoint(huggingfacehub_api_token=huggingface_api_key,model="Qwen/Qwen2.5-Coder-32B-Instruct")
# Create a chain
chain = prompt | llm
# Invoke the chain
output = chain.invoke({"book_name": "Deathly Hallows", "author": "J.K. Rowling"})
print(output[:100])

 "Deathly Hallows" is the final book in J.K. Rowling's beloved "Harry Potter" series. Published in 2


In [11]:
from langchain_core.prompts import ChatPromptTemplate

In [12]:
llm = HuggingFaceEndpoint(
    repo_id="Qwen/Qwen2.5-Coder-32B-Instruct",
    task="text-generation",
    max_new_tokens=512,
    do_sample=False,
    repetition_penalty=1.03,
)

chat_model = ChatHuggingFace(llm=llm)
template = ChatPromptTemplate([
   ('system', 'You are a helpful AI bot. Your specialty is {specialty}.'),
   ('human', 'Explain the concept of {concept} based on your expertise.')
])

In [13]:
specialties = ["psychology", "economics", "politics"]
concept = "time"
# Call the model with different personalities
for s in specialties:
   chain = template | chat_model
   output = chain.invoke({"specialty": s, "concept": concept})
   print(output.content[:100], end="\n" + "-" * 25 + '\n')

Certainly! In psychology, the concept of time is multifaceted and encompasses both how we perceive a
-------------------------
Certainly! While time is a fundamental concept in physics and philosophy, it also intersects with ec
-------------------------
Certainly! While my expertise primarily lies in politics, the concept of time is a fundamental aspec
-------------------------


In [27]:
!pip install -Uq arxiv  

In [14]:
from langchain_community.tools import ArxivQueryRun
tool = ArxivQueryRun()
print(tool.invoke('Photosynthesis')[:250])

Published: 2019-08-28
Title: Photosynthesis on Exoplanets and Exomoons from Reflected Light
Authors: Manasvi Lingam, Abraham Loeb
Summary: Photosynthesis offers a convenient means of sustaining biospheres. We
quantify the constraints for photosynthes


In [15]:
from langchain_community.agent_toolkits.load_tools import load_tools
from langchain.tools import Tool
from diffusers import StableDiffusionPipeline
import torch
import requests

# Step 1: Load Stable Diffusion Pipeline
model_id = "runwayml/stable-diffusion-v1-5"  # Stable Diffusion model on Hugging Face
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
#pipe = pipe.to("cuda")  # Use GPU for better performance; switch to "cpu" if no GPU

# Step 2: Define a custom tool for image generation
def generate_image_via_api(prompt: str) -> str:
    """
    Generate an image using the Hugging Face Inference API.
    
    Args:
        prompt (str): Text description of the image to generate.
        
    Returns:
        str: Path to the saved image file.
    """
    response = requests.post("https://api-inference.huggingface.co/models/stabilityai/stable-diffusion-xl-base-1.0", headers={"Authorization": f"Bearer {huggingface_api_key}"}, json={"inputs": prompt})
    
    if response.status_code == 200:
        # Save the image
        file_name = "generated_image.png"
        with open(file_name, "wb") as f:
            f.write(response.content)
        return f"Image saved at: {file_name}"
    else:
        return f"Failed to generate image: {response.json()}"

# Wrap it into a LangChain Tool
image_generation_tool = Tool(
    name="ImageGenerator",
    func=generate_image_via_api,
    description="Generate images from text prompts using Stable Diffusion."
)

# Step 3: Add LangChain Tools (you can combine this with other tools)
tools = load_tools(["arxiv"])  # Load LangChain's built-in tools
tools.append(image_generation_tool)  # Add our custom image generation tool

# Step 4: Use the tool in LangChain
prompt = "A futuristic cityscape at sunset, cyberpunk style"
result = image_generation_tool.func(prompt)
print(result)  # This will print the path to the generated image


Loading pipeline components...: 100%|██████████| 7/7 [00:27<00:00,  3.97s/it]


Image saved at: generated_image.png


In [16]:
from langchain_community.tools import WikipediaQueryRun  # pip install wikipedia
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import YouTubeSearchTool  # pip install youtube_search
from langchain_community.tools.openai_dalle_image_generation import (
   OpenAIDALLEImageGenerationTool
)
from langchain_community.utilities.dalle_image_generator import DallEAPIWrapper

In [19]:
wiki_api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=250)
wikipedia = WikipediaQueryRun(description="A tool to explain things in text format. Use this tool if you think the user’s asked concept is best explained through text.", api_wrapper=wiki_api_wrapper)
print(wikipedia.invoke("Mobius strip"))

Page: Möbius strip
Summary: In mathematics, a Möbius strip, Möbius band, or Möbius loop is a surface that can be formed by attaching the ends of a strip of paper together with a half-twist. As a mathematical object, it was discovered by Johann Benedi


In [20]:
model_id = "runwayml/stable-diffusion-v1-5"  # Stable Diffusion model on Hugging Face
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
#pipe = pipe.to("cuda")  # Use GPU for better performance; switch to "cpu" if no GPU

# Step 2: Define a custom tool for image generation
def generate_image_via_api(prompt: str) -> str:
    """
    Generate an image using the Hugging Face Inference API.
    
    Args:
        prompt (str): Text description of the image to generate.
        
    Returns:
        str: Path to the saved image file.
    """
    response = requests.post("https://api-inference.huggingface.co/models/stabilityai/stable-diffusion-xl-base-1.0", headers={"Authorization": f"Bearer {huggingface_api_key}"}, json={"inputs": prompt})
    
    if response.status_code == 200:
        # Save the image
        file_name = "generated_image.png"
        with open(file_name, "wb") as f:
            f.write(response.content)
        return f"Image saved at: {file_name}"
    else:
        return f"Failed to generate image: {response.json()}"

# Wrap it into a LangChain Tool
image_generation_tool = Tool(
    name="ImageGenerator",
    func=generate_image_via_api,
    description="Generate images from text prompts using Stable Diffusion."
)


Loading pipeline components...: 100%|██████████| 7/7 [00:26<00:00,  3.82s/it]


In [23]:
youtube = YouTubeSearchTool(
   description="A tool to search YouTube videos. Use this tool if you think the user’s asked concept can be best explained by watching a video."
)
youtube.run("Oiling a bike's chain")

"['https://www.youtube.com/watch?v=X1Vze17bhgk&pp=ygUVT2lsaW5nIGEgYmlrZSdzIGNoYWlu', 'https://www.youtube.com/shorts/pFMRevCZXvM']"

In [24]:
tools = [wikipedia, image_generation_tool, youtube]

In [25]:
llm = HuggingFaceEndpoint(
    repo_id="Qwen/Qwen2.5-Coder-32B-Instruct",
    task="text-generation",
    max_new_tokens=512,
    do_sample=False,
    repetition_penalty=1.03,
)

chat_model = ChatHuggingFace(llm=llm)
model_with_tools = chat_model.bind_tools(tools)

In [26]:
response = model_with_tools.invoke([HumanMessage("What's up?!")])
print(f"Text response: {response.content}")
print(f"Tools used in the response: {response.tool_calls}")

Text response: Hello! Not much, just here to help you out. How can I assist you today?
Tools used in the response: []


In [27]:
response = model_with_tools.invoke([
   HumanMessage("Can you generate an image of a mountain bike?")
])
print(f"Text response: {response.content}")
print(f"Tools used in the response: {response.tool_calls}")

Text response: 
Tools used in the response: [{'name': 'ImageGenerator', 'args': {'__arg1': 'generate an image of a mountain bike'}, 'id': '0', 'type': 'tool_call'}]


In [28]:
from langgraph.prebuilt import create_react_agent
system_prompt = SystemMessage("You are a helpful bot named Chandler.")
agent = create_react_agent(chat_model, tools, state_modifier=system_prompt)

In [29]:
from pprint import pprint
response = agent.invoke({"messages": HumanMessage("What's up?")})
pprint(response["messages"])

[HumanMessage(content="What's up?", additional_kwargs={}, response_metadata={}, id='1fe99b20-d73e-4607-b6f9-e815195555f1'),
 AIMessage(content='Hello! Not much, just here to help you out. How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': ChatCompletionOutputUsage(completion_tokens=35, prompt_tokens=382, total_tokens=417), 'model': '', 'finish_reason': 'stop'}, id='run-31738121-a062-4b74-90d4-e94da2520177-0')]


In [30]:
response = agent.invoke({"messages": [
   HumanMessage('Explain how photosynthesis works.')
]})
print(len(response['messages']))

18


In [31]:
for message in response['messages']:
   print(
       f"{message.__class__.__name__}: {message.content}"
   )  # Print message class name and its content
   print("-" * 20, end="\n")

HumanMessage: Explain how photosynthesis works.
--------------------
AIMessage: 
--------------------
ToolMessage: Page: Photosynthesis
Summary: Photosynthesis ( FOH-tə-SINTH-ə-sis) is a system of biological processes by which photosynthetic organisms, such as most plants, algae, and cyanobacteria, convert light energy, typically from sunlight, into the chemical 
--------------------
AIMessage: 
--------------------
ToolMessage: Page: Photosynthesis
Summary: Photosynthesis ( FOH-tə-SINTH-ə-sis) is a system of biological processes by which photosynthetic organisms, such as most plants, algae, and cyanobacteria, convert light energy, typically from sunlight, into the chemical 
--------------------
AIMessage: 
--------------------
ToolMessage: Page: Photosynthesis
Summary: Photosynthesis ( FOH-tə-SINTH-ə-sis) is a system of biological processes by which photosynthetic organisms, such as most plants, algae, and cyanobacteria, convert light energy, typically from sunlight, into the chemical

In [32]:
def execute(agent, query):
   response = agent.invoke({'messages': [HumanMessage(query)]})
  
   for message in response['messages']:
       print(
           f"{message.__class__.__name__}: {message.content}"
       )  # Print message class name and its content
      
       print("-" * 20, end="\n")
  
   return response

In [33]:
system_prompt = SystemMessage(
   """
   You are a helpful bot named Chandler. Your task is to explain topics
   asked by the user via three mediums: text, image or video.
  
   If the asked topic is best explained in text format, use the Wikipedia tool.
   If the topic is best explained by showing a picture of it, generate an image
   of the topic using Dall-E image generator and print the image URL.
   Finally, if video is the best medium to explain the topic, conduct a YouTube search on it
   and return found video links.
   """
)

In [34]:
agent = create_react_agent(chat_model, tools, state_modifier=system_prompt)
response = execute(agent, query='Explain the Fourier Series visually.')

HumanMessage: Explain the Fourier Series visually.
--------------------
AIMessage: 
--------------------
ToolMessage: Image saved at: generated_image.png
--------------------
AIMessage: 
--------------------
ToolMessage: ['https://www.youtube.com/watch?v=spUNpyF58BY&t=477s&pp=ygUcRm91cmllciBTZXJpZXMgdmlzdWFsaXphdGlvbg%3D%3D', 'https://www.youtube.com/watch?v=ds0cmAV-Yek&pp=ygUcRm91cmllciBTZXJpZXMgdmlzdWFsaXphdGlvbg%3D%3D']
--------------------
AIMessage: 
--------------------
ToolMessage: ['https://www.youtube.com/watch?v=spUNpyF58BY&t=477s&pp=ygUcRm91cmllciBTZXJpZXMgdmlzdWFsaXphdGlvbg%3D%3D', 'https://www.youtube.com/watch?v=DY8AAsRrFUM&pp=ygUcRm91cmllciBTZXJpZXMgdmlzdWFsaXphdGlvbg%3D%3D']
--------------------
AIMessage: 
--------------------
ToolMessage: ['https://www.youtube.com/watch?v=spUNpyF58BY&t=477s&pp=ygUcRm91cmllciBTZXJpZXMgdmlzdWFsaXphdGlvbg%3D%3D', 'https://www.youtube.com/watch?v=ds0cmAV-Yek&pp=ygUcRm91cmllciBTZXJpZXMgdmlzdWFsaXphdGlvbg%3D%3D']
--------------------
AIMess

In [35]:
response = execute(agent, query="What did I ask you in the previous query?")

HumanMessage: What did I ask you in the previous query?
--------------------
AIMessage: 
--------------------
ToolMessage: Page: How to Train Your Dragon (novel series)
Summary: How to Train Your Dragon is a series of children's books written by British author Cressida Cowell. The books are set in a fictional Fantasy Viking world, and focus on the experiences of protagon
--------------------
AIMessage: 
--------------------
ToolMessage: Page: How to Train Your Dragon (novel series)
Summary: How to Train Your Dragon is a series of children's books written by British author Cressida Cowell. The books are set in a fictional Fantasy Viking world, and focus on the experiences of protagon
--------------------
AIMessage: 
--------------------
ToolMessage: Page: How to Train Your Dragon (novel series)
Summary: How to Train Your Dragon is a series of children's books written by British author Cressida Cowell. The books are set in a fictional Fantasy Viking world, and focus on the experiences of

In [ ]:
from langgraph.checkpoint.sqlite import SqliteSaver
memory = SqliteSaver.from_conn_string(':agent_history:')
agent = create_react_agent(chat_model, tools, checkpointer=memory, state_modifier=system_prompt)